In [1]:
import sys

sys.path.append("../")

from datasets import load_dataset
from transformers import AutoTokenizer
from src.data import *
from accelerate import accelerator

In [2]:
dataset = RawMTDataset.wmt16_ro_en()
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
tokenized_dataset = TokenizedMTDataset(dataset, tokenizer)

Reusing dataset wmt16 (/Users/akurniawan/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/0d9fb3e814712c785176ad8cdb9f465fbe6479000ee6546725db30ad8a8b5f8a)
100%|██████████| 3/3 [00:00<00:00, 57.78it/s]


In [7]:
# tokenizer("this is brazil")
tokenizer.vocab["[CLS]"]

101

In [30]:
preprocessed_dataset = tokenized_dataset.preprocess(8)

Loading cached processed dataset at /Users/akurniawan/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/0d9fb3e814712c785176ad8cdb9f465fbe6479000ee6546725db30ad8a8b5f8a/cache-31a41103e19cab6f.arrow
Loading cached processed dataset at /Users/akurniawan/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/0d9fb3e814712c785176ad8cdb9f465fbe6479000ee6546725db30ad8a8b5f8a/cache-65d66b5e8ff99eaa.arrow
Loading cached processed dataset at /Users/akurniawan/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/0d9fb3e814712c785176ad8cdb9f465fbe6479000ee6546725db30ad8a8b5f8a/cache-313bfa5a8bb711d4.arrow
Loading cached processed dataset at /Users/akurniawan/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/0d9fb3e814712c785176ad8cdb9f465fbe6479000ee6546725db30ad8a8b5f8a/cache-4e8c0a3acd489813.arrow
Loading cached processed dataset at /Users/akurniawan/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/0d9fb3e814712c785176ad8cdb9f465fbe6479000ee6546725db30ad8a8b5f8a/cache-faeee1de00e0c678.arrow
Loading cached processed datas

In [31]:
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-multilingual-cased")
model.add_adapter("testing")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [32]:
from transformers import BertLMHeadModel, BertConfig

config = BertConfig(vocab_size=119547, is_decoder=True, add_cross_attention=True)
model_lm = BertLMHeadModel.from_pretrained("bert-base-multilingual-cased", config=config)
model_lm.add_adapter("testing")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['bert.encoder.layer.4.crossattention.self.query.weight', 'bert.encoder.layer.6.crossattention.self.query.weight', 'bert.encoder.layer.10.crossattention.self.key.weight', 'bert.encoder.layer.7.crossattention.self.query

In [27]:
from transformers import EncoderDecoderModel
model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-multilingual-cased', 'bert-base-multilingual-cased')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.wei

In [28]:
# model.add_adapter("testing")
# model.encoder.encoder.layer[0].output.adapters["testing"].adapter_down[0].weight

In [29]:
# model.decoder.bert.encoder.layer[0].output.adapters["testing"].adapter_down[0].weight

In [30]:
model.encoder.add_adapter("testing")
model.train_adapter("testing")
# model.active_adapters
for k, v in model.named_parameters():
    if "adapters" in k or "crossattention" in k:
        print(k, v.requires_grad)

encoder.encoder.layer.0.output.adapters.testing.adapter_down.0.weight True
encoder.encoder.layer.0.output.adapters.testing.adapter_down.0.bias True
encoder.encoder.layer.0.output.adapters.testing.adapter_up.weight True
encoder.encoder.layer.0.output.adapters.testing.adapter_up.bias True
encoder.encoder.layer.1.output.adapters.testing.adapter_down.0.weight True
encoder.encoder.layer.1.output.adapters.testing.adapter_down.0.bias True
encoder.encoder.layer.1.output.adapters.testing.adapter_up.weight True
encoder.encoder.layer.1.output.adapters.testing.adapter_up.bias True
encoder.encoder.layer.2.output.adapters.testing.adapter_down.0.weight True
encoder.encoder.layer.2.output.adapters.testing.adapter_down.0.bias True
encoder.encoder.layer.2.output.adapters.testing.adapter_up.weight True
encoder.encoder.layer.2.output.adapters.testing.adapter_up.bias True
encoder.encoder.layer.3.output.adapters.testing.adapter_down.0.weight True
encoder.encoder.layer.3.output.adapters.testing.adapter_down.